In [135]:
# inladen benodigde packages
import pandas as pd
import math
import itertools

seed = 9

In [145]:
# inladen dataset met kolom voor scores
# inladen data
dataset_path = "~/share/Milena/programma-discriminatie-en-racisme/01_datapreparatie/"
dataset_empty = pd.read_csv(dataset_path + 'dataset_dummy_empty.csv', sep = ';') # voor dummy dataset importeer eerste paar instances
dataset_empty.rename(columns={"Unnamed: 0": "nummer"})
display(dataset_empty)

,Unnamed: 0,id,document,text,zin,score
0,0,d4cfb963-de8d-400e-bb87-5493b794a255_1_4adc737...,Beslisnota bij Kamerbrief over Geografische Ba...,...,...,NaN
1,1,d4cfb963-de8d-400e-bb87-5493b794a255_2_4adc737...,Beslisnota bij Kamerbrief over Geografische Ba...,...,Onze referentie\nVerdel...,NaN
2,2,7eb69333-4557-4594-b5be-1276e3c0f2da_2_027bdd4...,Kwartaalrapportage Q4 2023,Onze referentie\nVerdel...,"groeifaciliteit voor de 23987, nr. aanspraa...",NaN
3,3,ad741e24-86a7-47a2-ad12-fbde149fe4bc_8_e4269c3...,Advies GR; Verbetermogelijkheden voor het bevo...,"groeifaciliteit voor de 23987, nr. aanspraa...",hoofdstuk 01 | Inleiding ...,NaN
4,4,ad741e24-86a7-47a2-ad12-fbde149fe4bc_17_e4269c...,Advies GR; Verbetermogelijkheden voor het bevo...,hoofdstuk 01 | Inleiding ...,"hoofdstuk 03 | Effectiviteit, resultaten, nade...",NaN
...,...,...,...,...,...,...
6328,6328,490faff2-b50c-4a78-afdc-f36ed34ed3df_10_039b80...,"Quick scan Financiële impact maatregelen ""Voor...",QUICK SCAN D.D...,QUICK SCAN D....,NaN
6329,6329,490faff2-b50c-4a78-afdc-f36ed34ed3df_11_039b80...,"Quick scan Financiële impact maatregelen ""Voor...",QUICK SCAN D....,QUICK SCAN...,NaN
6330,6330,490faff2-b50c-4a78-afdc-f36ed34ed3df_15_039b80...,"Quick scan Financiële impact maatregelen ""Voor...",QUICK SCAN...,QUICK SCA...,NaN
6331,6331,490faff2-b50c-4a78-afdc-f36ed34ed3df_16_039b80...,"Quick scan Financiële impact maatregelen ""Voor...",QUICK SCA...,QUICK...,NaN


In [146]:
def split_annotated_data(data, n_annotators, n_annotations = 3):
    """ functie die de data splitst. iedere zin wordt n_annotations keer toegevoegd, 
    waarbij er wordt doorgeteld aan het aantal annotators, waardoor iedere annotator (ongeveer) evenveel annnotations ontvangt.
    De data kan na het annotaten weer gemerged worden door middel van de functie merge_annotated_data.
    
    Input:
    data: een DataFrame die gesplit moet worden
    n_annotators: int, het aantal annotators. Moet groter zijn dan n_annotations
    n_annotations: int, het aantal annotations per zin. default 3
    
    Output:
    datasets: een lijst van n_annotators DataFrames.
    """
    
    # creëer lege list of dfs voor de uiteindelijke output
    datasets = []
    
    for i in range(n_annotators):
        df = pd.DataFrame()
        #df[f"score_{i+1}"] = ''
        datasets.append(df)
    
    # DataFrame husselen om een beetje randomisatie in te bouwen
    data = data.sample(frac = 1)
    
    # bouw lijst met unieke combinaties van n_annotators met len = n_annotations
    for i in range(n_annotations):
        cycle = itertools.cycle("".join(f"{(j+i)%n_annotators+1}" for j in range(n_annotators)))
        
        # maak lijsten even lang als data en voeg toe als nieuwe kolom
        list_i = [next(cycle) for x in range(len(data))]
        data[f"labeller_{str(i+1)}"] = list_i
    
    # sorteer op labeller
    for i in range(n_annotations):
        data = data.sort_values(by=[f"labeller_{str(i+1)}"])
        for j in range(n_annotators):
            subset = data[data[f"labeller_{str(i+1)}"] == str(j+1)]
            datasets[j] = pd.concat([datasets[j], subset])
    
    return datasets

split_data = split_annotated_data(data = dataset_empty, n_annotators = 5, n_annotations = 3)
#display(split_data)

In [147]:
# output genereren: iedere dataset naar .csv
for i, dataset in enumerate(split_data):
    dataset.to_csv('te_labellen_' + str(i) + '.csv')

In [32]:
### OUDE CODE ###
def split_annotated_data(data, n_annotators, n_annotations):
    """ functie die de data splitst. iedere zin wordt n_annotations keer toegevoegd, 
    waarbij er wordt doorgeteld aan het aantal annotators, waardoor iedere annotator (ongeveer) evenveel annnotations ontvangt.
    De data kan na het annotaten weer gemerged worden door middel van de functie merge_annotated_data.
    
    Input:
    data: een DataFrame die gesplit moet worden
    n_annotators: int, het aantal annotators
    n_annotations: int, het aantal annotations per zin
    
    Output:
    datasets: een lijst van n_annotators DataFrames.
    """
    # creëer lege list of lists voor de uiteindelijke output
    datasets = []
    for i in range(n_annotators):
        datasets.append([])
    
    # variabelen om bij te houden waar we zijn in het toevoegen. Een zin wordt n_annotations keer toegevoegd, 
    # beginnend bij datasets
    counter = 0
    max_counter = n_annotators -1
    
    # datapunt creëren
    for i in range(len(data)):
        datapunt = data.iloc[i]
        # het datapunt aan de nodige dataset toevoegen
        for j in range(n_annotations):
            if counter < max_counter:
                datasets[counter].append(datapunt)
                counter += 1
            else: # counter = n_annotators
                datasets[counter].append(datapunt)
                counter = 0
    
    # lijsten converteren naar df zodat ze naar .csv te converteren zijn
    for i in range(n_annotators):
        datasets[i] = pd.DataFrame(datasets[i])
    
    return datasets
    
# output genereren en kijken naar de lengte
split_data = split_annotated_data(data = dataset_empty, n_annotators = 5, n_annotations = 3)
print("Lengte te labelen per persoon na splitten:", len(split_data[4]))
print("Lengte originele dataset:", len(dataset_empty))

Lengte te labelen per persoon na splitten: 3799
Lengte originele dataset: 6333
